# Data exploration

# Purpose
Explore the ForSea data

# Methodology
Quickly describe assumptions and processing steps.

# Setup

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import src.data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
